# NLP HW 2

## Дарья Родионова БКЛ182

### Importing packages

In [1]:
from sklearn.metrics import accuracy_score
import re

import pymorphy2
m = pymorphy2.MorphAnalyzer()
from pymorphy2.tokenizers import simple_word_tokenize

from natasha import(
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

import json
from pymystem3 import Mystem
ms = Mystem()

import spacy

import nltk
from nltk import word_tokenize

import flair
from flair.data import Sentence
from flair.models import SequenceTagger

### Texts

<p> В русском тексте будут проблемы с нетипичными написаниями некоторых слов (сахару, иль). Кроме того, pos-tagger может распознать обращения (милая, нежная) и имена собственные (Великолепный) как прилагательные, а также некоторые существительные как императивы (черни) и наоборот (плачь). <p>

В английском тексте, скорее всего, возникнут сложности с сокращениями (I've, etc.), фразовыми глаголами (makes up), также будут сложности в разметке званий (General как прилагательное), слов с дефисами (so-called) и восклицаниями (alas). 

In [120]:
rus_text = '''Моя любовь к тебе сейчас — слонёнок, 
родившийся в Берлине иль Париже и топающий ватными ступнями по комнатам хозяина зверинца. 
Не предлагай ему французских булок, он может съесть лишь дольку мандарина,
кусочек сахару или конфету. Не плачь, о нежная, что в тесной клетке он сделается посмеяньем черни,
чтоб в нос ему пускали дым сигары приказчики под хохот мидинеток. Не думай, милая, что день настанет,
когда, взбесившись, разорвет он цепи и побежит по улицам и будет, как автобус, давить людей вопящих.
Нет, пусть тебе приснится он под утро в парче и меди, в страусовых перьях, как тот, 
Великолепный, что когда-то нес к трепетному Риму Ганнибала.'''

eng_text = '''The last time the US was invaded was when General Francisco Villa attacked Columbus, 
New Mexico in 1916, and alas Pancho is no more, having been assassinated in his black Dodge Roadster.
Our sociocultural perception of wolves is influenced by the media, 
and millennia-old ideas about wolves being a threat to livestock, that they’re evil, etc.
But I’ve always had a financial commitment on top of my tithe that is not legally classified as rent.
Well I’m curious, what is this so-called accusation against six-pointer?
We’re saving so much by paying for those in cash it more than makes up for the interest payments going to the car loan this year.'''

In [3]:
rus_tagged = ['Моя_PRON', 'любовь_NOUN', 'к_PREP', 'тебе_PRON', 'сейчас_ADV', 'слонёнок_NOUN',
              'родившийся_ADJ', 'в_PREP', 'Берлине_NOUN', 'иль_CONJ', 'Париже_NOUN',
              'и_CONJ', 'топающий_ADJ', 'ватными_ADJ', 'ступнями_NOUN',
              'по_PREP', 'комнатам_NOUN', 'хозяина_NOUN', 'зверинца_NOUN',
              'Не_PART', 'предлагай_VERB', 'ему_PRON', 'французских_ADJ', 'булок_NOUN',
              'он_PRON', 'может_VERB', 'съесть_VERB', 'лишь_PART', 'дольку_NOUN', 
              'мандарина_NOUN', 'кусочек_NOUN', 'сахару_NOUN', 'или_CONJ', 'конфету_NOUN',
              'Не_PART', 'плачь_VERB', 'о_PREP', 'нежная_NOUN', 'что_CONJ', 
              'в_PREP', 'тесной_ADJ', 'клетке_NOUN',
              'он_PRON', 'сделается_VERB', 'посмеяньем_NOUN', 'черни_NOUN',
              'чтоб_PREP', 'в_PREP', 'нос_NOUN', 'ему_PRON', 'пускали_VERB', 'дым_NOUN', 'сигары_NOUN',
              'приказчики_NOUN', 'под_PREP', 'хохот_NOUN', 'мидинеток_NOUN',
              'Не_CONJ', 'думай_VERB', 'милая_NOUN', 'что_CONJ', 'день_NOUN', 'настанет_VERB',
              'когда_CONJ', 'взбесившись_VERB', 'разорвет_VERB', 'он_PRON', 'цепи_NOUN',
              'и_VERB', 'побежит_VERB', 'по_PREP', 'улицам_NOUN', 'и_CONJ', 'будет_VERB',
              'как_CONJ', 'автобус_NOUN', 'давить_VERB', 'людей_NOUN', 'вопящих_ADJ',
              'Нет_PART', 'пусть_PART', 'тебе_PRON', 'приснится_VERB', 'он_PRON', 'под_PRON', 'утро_NOUN',
              'в_PREP', 'парче_NOUN', 'и_CONJ', 'меди_NOUN', 'в_PREP', 'страусовых_ADJ', 'перьях_NOUN',
              'как_CONJ', 'тот_PRON', 'Великолепный_NOUN', 'что_CONJ', 
              'когда-то_ADV', 'нес_VERB', 'к_PREP', 'трепетному_ADJ', 'Риму_NOUN', 'Ганнибала_NOUN']

In [4]:
len(rus_tagged)

103

In [5]:
eng_tagged = ['The_ART', 'last_ADJ', 'time_NOUN', 'the_ART', 'US_NOUN', 'was_VERB', 'invaded_VERB', 'was_VERB',
              'when_CONJ', 'General_NOUN', 'Francisco_NOUN', 'Villa_NOUN', 'attacked_VERB', 'Columbus_NOUN', 
              'New_NOUN', 'Mexico_NOUN', 'in_PREP', '1916_NUM', 'and_CONJ', 'alas_INTJ', 'Pancho_NOUN', 
              'is_VERB', 'no_PART', 'more_ADV', 'having_VERB', 'been_VERB', 'assassinated_VERB', 'in_PREP', 
              'his_PRON', 'black_ADJ', 'Dodge_NOUN', 'Roadster_NOUN', 'Our_PRON', 'sociocultural_ADJ', 
              'perception_NOUN', 'of_PREP', 'wolves_NOUN', 'is_VERB', 'influenced_VERB', 'by_PREP', 'the_ART', 
              'media_NOUN', 'and_CONJ', 'millennia-old_ADJ', 'ideas_NOUN', 'about_PREP', 'wolves_NOUN', 
              'being_VERB', 'a_ART', 'threat_NOUN', 'to_PART', 'livestock_NOUN', 'that_ADV', 'they_PRON', 
              're_VERB', 'evil_ADJ', 'etc_ART', 'But_CONJ', 'I_PRON', 've_VERB', 'always_ADV', 'had_VERB', 
              'a_ART', 'financial_ADJ', 'commitment_NOUN', 'on_PREP', 'top_NOUN', 'of_PREP', 'my_PRON', 
              'tithe_NOUN', 'that_ADV', 'is_VERB', 'not_ADV', 'legally_ADV', 'classified_VERB', 'as_ADV', 'rent_NOUN', 
              'Well_INTJ', 'I_PRON', 'm_VERB', 'curious_ADJ', 'what_CONJ', 'is_VERB', 'this_PRON', 
              'so-called_ADJ', 'accusation_NOUN', 'against_PREP', 'six-pointer_NOUN', 'We_PRON', 're_VERB', 
              'saving_VERB', 'so_ADV', 'much_ADJ', 'by_PREP', 'paying_VERB', 'for_PREP', 'those_PRON', 
              'in_PREP', 'cash_NOUN', 'it_PRON', 'more_ADV', 'than_ADV', 'makes_VERB', 'up_PART', 'for_PREP', 
              'the_ART', 'interest_NOUN', 'payments_NOUN', 'going_VERB', 'to_PART', 'the_ART', 
              'car_NOUN', 'loan_NOUN', 'this_PRON', 'year_NOUN']

In [6]:
len(eng_tagged)

115

Функция ниже соотносит предписанные теги с правильной разметкой для обоих языков.

In [15]:
rus_pos = {'LATN': 'NOUN', 'PROPN': 'PRON', 'S': 'NOUN',
           'ADJF': 'ADJ', 'ADJS': 'ADJ', 'PRTF': 'ADJ', 'PRTS': 'ADJ', 'A': 'ADJ',
           'ADVB': 'ADV', 'COMP': 'ADV', 'PRED': 'ADV',
           'INFN': 'VERB', 'GRND': 'VERB', 'AUX': 'VERB', 'V': 'VERB',
           'CCONJ': 'CONJ', 'SCONJ': 'CONJ', 'ADP': 'PREP', 'PR': 'PREP',
           'NPRO': 'PRON', 'DET': 'PRON', 'ADVPRO': 'PRON', 'SPRO': 'PRON', 'APRO': 'PRON',
           'PRCL': 'PART', 'ADP': 'PREP', 'PR': 'PREP'} 

def set_pos_rus(pos, dictionary=rus_pos):
    if pos in dictionary:
        return dictionary[pos]
    else:
        return pos

In [76]:
eng_pos = {'NN': 'NOUN', 'NNP': 'NOUN', 'NNS': 'NOUN', 'PDT': 'PRON', 'PRP': 'PRON', 'PRP$': 'PRON', 
           'PROPN': 'NOUN', 'VBG': 'VERB', 'VBN': 'VERB', 'VBP': 'VERB', 'VBZ': 'VERB', 'AUX': 'VERB',
           'CC': 'CONJ', 'CD': 'NUMB', 'DT': 'ART', 'IN': 'PREP', 'JJ': 'ADJ', 'JJS': 'ADJ', 'MD': 'VERB',
           'RB': 'ADV', 'RP': 'PART', 'TO': 'PART', 'UH': 'INTJ', 'DET': 'ART',  'NUM': 'NUMB', 'WRB': 'ADV',
           'CCONJ': 'CONJ', 'SCONJ': 'CONJ', 'VB': 'VERB', 'ADP': 'PREP',  }

def set_pos_eng(pos, dictionary=eng_pos):
    if pos in dictionary:
        return dictionary[pos]
    else:
        return pos

### PyMorphy

In [17]:
def accuracy_check(predicted, gold):
    print('Accuracy: %.4f' % accuracy_score(predicted, gold))

In [18]:
rus_tokens = simple_word_tokenize(rus_text)
pymorphy_result = []

In [19]:
punct = "'.,!?:’—»«"

In [20]:
for token in rus_tokens:
    if token in punct:
        continue
    gram = m.parse(token)[0]
    pos = gram.tag.POS
    if not pos:
        pos = gram.tag

    p = str(set_pos_rus(pos))
    pymorphy_result.append(token + '_' + p)

In [48]:
len(pymorphy_result)
# pymorphy_result

103

In [22]:
accuracy_check(pymorphy_result, rus_tagged)

Accuracy: 0.9126


### Natasha

In [23]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
doc = Doc(rus_text)

In [24]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [25]:
natasha_result = [i.text + '_' + set_pos_rus(i.pos) for i in doc.tokens if i.text not in punct]

In [47]:
len(natasha_result)
# natasha_result

103

In [27]:
accuracy_check(natasha_result, rus_tagged)

Accuracy: 0.8155


### MyStem

In [28]:
mystem_result = []

In [29]:
for token in rus_tokens:
    if token in punct:
        continue
    text = ms.analyze(token)
    pos = ''
    for word in text:
        if 'analysis' in word:
            gram = word.get('analysis')[0]
            pos = gram['gr']
            pos = pos.split('=')[0]
            pos = pos.split(',')[0]
                    
            p = set_pos_rus(pos)
            mystem_result.append(token + '_' + p)

In [45]:
len(mystem_result)
# mystem_result

103

In [46]:
accuracy_check(mystem_result, rus_tagged)

Accuracy: 0.7573


Как мы видим, лучше всего разметил PyMorphy. 

### SpaCy

In [136]:
nlp = spacy.load('en_core_web_sm')

In [137]:
spacy_result = []
for token in eng_tokens:
    if token in punct:
        continue
    pos = ''
    for t in nlp(token):
        p = t.pos_
        if pos == '':
            pos = p
        else:
            break
    
    spacy_result.append(token + '_' + set_pos_eng(pos, eng_pos))

In [138]:
len(spacy_result)
# spacy_result

115

In [139]:
accuracy_check(spacy_result, eng_tagged)

Accuracy: 0.6783


### NLTK

In [140]:
# nltk.download('punkt')

In [141]:
# nltk.download('averaged_perceptron_tagger')

In [142]:
nltk_result = [i[0] + '_' + set_pos_eng(i[1], eng_pos) for i in nltk.pos_tag(eng_tokens) if i[0] not in punct]

In [143]:
len(nltk_result)
# nltk_result

115

In [144]:
accuracy_check(nltk_result, eng_tagged)

Accuracy: 0.7826


### Flair

In [89]:
pos = SequenceTagger.load('pos-fast')

2020-10-18 16:16:24,393 https://nlp.informatik.hu-berlin.de/resources/models/pos-fast/en-pos-ontonotes-fast-v0.5.pt not found in cache, downloading to /var/folders/b_/1gk5pbbs15b65fy6445mbvqh0000gn/T/tmpklxpqxl7


100%|██████████| 75266317/75266317 [00:19<00:00, 3870324.74B/s]

2020-10-18 16:16:44,137 copying /var/folders/b_/1gk5pbbs15b65fy6445mbvqh0000gn/T/tmpklxpqxl7 to cache at /Users/ddrodionova/.flair/models/en-pos-ontonotes-fast-v0.5.pt


2020-10-18 16:16:44,172 removing temp file /var/folders/b_/1gk5pbbs15b65fy6445mbvqh0000gn/T/tmpklxpqxl7
2020-10-18 16:16:44,176 loading file /Users/ddrodionova/.flair/models/en-pos-ontonotes-fast-v0.5.pt


In [134]:
eng_text = re.sub(r'[^-’\w\s]','', eng_text)
for s in eng_text.split('\n'):
    tagged_text = []
    text = Sentence(eng_text)
    pos.predict(text)
    tagged_text.append(re.sub(' <', '<', text.to_tagged_string()))

In [133]:
# tagged_text

In [127]:
flair_result = []
for pair in tagged_text:
    part = pair.split(' ')
    result = [i.split('<')[0] + '_' + set_pos_eng(re.search('<([^>]+)>', i).group(1), eng_pos) for i in part]
    flair_result += result

In [131]:
len(flair_result)
# flair_result

115

In [132]:
accuracy_check(flair_result, eng_tagged)

Accuracy: 0.7913


In [135]:
eng_tokens = word_tokenize(eng_text)

Вторая часть задания находится в файле NLP_HW2_2. 